# A brief summary of the Standard Library in C++ (98)
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [Design Approach](#design-approach)

<hr>

The C++ standard library resides in the namespace `std`.

* Header files are included as `<header-name>`, omitting the `.h` extension. For example, `<vector>`.

* Header files from the C standard library may be included as `<c-header-name>`. For example, `<cstdio>`.

* Header files from the C standard library may be included directly into the global namespace as `<header-name.h>`. For example, `<stdio.h>`.

* User-defined header files may be included as `"header-name.h"`. For example, `"myvector.h"`.

## Design Approach <a class="anchor" id="design-approach"></a>

The standard library makes extensive use of templates. There is no common templates.

In [ ]:
int m